<a href="https://colab.research.google.com/github/nicchic/NLP/blob/main/mental_gpa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("Amod/mental_health_counseling_conversations")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    context_tokens = tokenizer(examples['Context'], padding="max_length", truncation=True, max_length=128)
    response_tokens = tokenizer(examples['Response'], padding="max_length", truncation=True, max_length=128)

    return {
        "context_input_ids": context_tokens["input_ids"],
        "context_attention_mask": context_tokens["attention_mask"],
        "response_input_ids": response_tokens["input_ids"],
        "response_attention_mask": response_tokens["attention_mask"]
    }

In [ ]:
tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
attention_mask = torch.tensor(tokenized_ds['train']['context_attention_mask'][0]).unsqueeze(0).to(device)


In [ ]:
input_ids = torch.tensor(tokenized_ds['train']['context_input_ids'][0]).unsqueeze(0).to(device)

output = model.generate(input_ids,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        max_length=300,
                        do_sample=True,
                        top_p=0.9
                        )

generated_text_top = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text (Top-p Sampling):")
print(generated_text_top)


Generated Text (Top-p Sampling):
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?1 This is a question I'm going to get to in an hour.   I'm going to go through some things with my feelings and myself.   I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I don't see myself as an average guy, I just feel like a loser. I don't feel like I belong here.
   I feel that I'm worth nothing. I don't know what to do with my life, I can't think for myself. I am not alone, I'm an independent, caring and compassionate person.
   This situation has impacted me in many ways, many different ways and with different levels of trauma.   I feel like I am an outsider. I feel like I have

In [ ]:
output_beam = model.generate(input_ids,
                        max_length= 300,
                        attention_mask=attention_mask,
                        pad_token_id=tokenizer.eos_token_id,
                        num_beams=5,
                        do_sample= False,
                        no_repeat_ngram_size = 3
                        )

generated_text_beam = tokenizer.decode(output_beam[0], skip_special_tokens=True)
print("\nGenerated Text (Beam Search):")
print(generated_text_beam)


Generated Text (Beam Search):
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?The only way I can change my feelings is by doing something about it. I don't know how to do that, but it's something I can do. It's something that I've been doing for a long time, and I'm not going to let it stop me from doing what I want to do. I'm going to do what I feel is right for me, and it's going to be good for me.


Evaluation

In [ ]:
pip install rouge_score

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

In [ ]:
reference_text =  "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.  How can I change my feeling of being worthless to everyone? Here are few things that you could do to make a positive change of you feel about yourself.List all the things that you want to do in life and then start working on one thing at a time. Give yourself credit for even the smallest accomplishment and don't forget to celebrate the fact that you've tried."


In [ ]:
rouge_result_top_p = rouge.compute(predictions=[generated_text_top], references=[reference_text])

print("\nROUGE Scores (Top-p Sampling):")

def display_rouge_scores(rouge_scores):
    for rouge_type, score in rouge_scores.items():
        print(f"{rouge_type.upper()}: {score:.4f}")

# Display the ROUGE scores
display_rouge_scores(rouge_result_top_p)



ROUGE Scores (Top-p Sampling):
ROUGE1: 0.6067
ROUGE2: 0.4906
ROUGEL: 0.5468
ROUGELSUM: 0.4494


In [ ]:
rouge_result_beam = rouge.compute(predictions=[generated_text_beam], references=[reference_text])

print("\nROUGE Scores (Beam Search):")
display_rouge_scores(rouge_result_beam)


ROUGE Scores (Beam Search):
ROUGE1: 0.6343
ROUGE2: 0.5038
ROUGEL: 0.5522
ROUGELSUM: 0.4552
